<a href="https://colab.research.google.com/github/Matheus-Homem/Matheus-Homem/blob/main/Future_Sales_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Imports

## 0.1. Libraries

In [88]:
import pandas as pd
import numpy as np
!pip install goslate
import goslate
import seaborn as sns
sns.set_style('darkgrid')


In [89]:
import warnings
warnings.filterwarnings('ignore')

## 0.2. Loading Data

In [90]:
# Importing data from .csv
item_categories = pd.read_csv("https://github.com/Matheus-Homem/future-sales-prediction/raw/main/data/item_categories.csv")
items = pd.read_csv("https://github.com/Matheus-Homem/future-sales-prediction/raw/main/data/items.csv")
sales_train = pd.read_csv("https://github.com/Matheus-Homem/future-sales-prediction/raw/main/data/sales_train.csv")
shops = pd.read_csv("https://github.com/Matheus-Homem/future-sales-prediction/raw/main/data/shops.csv")

In [91]:
# First row of 'item_categories'
item_categories.loc[0]

item_category_name    PC - Гарнитуры/Наушники
item_category_id                            0
Name: 0, dtype: object

In [92]:
# First row of 'items'
items.loc[0]

item_name           ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D
item_id                                                     0
item_category_id                                           40
Name: 0, dtype: object

In [93]:
# First row of 'sales_train'
sales_train.loc[0]

date              02.01.2013
date_block_num             0
shop_id                   59
item_id                22154
item_price               999
item_cnt_day               1
Name: 0, dtype: object

In [94]:
# First row of 'shops'
shops.loc[0]

shop_name    !Якутск Орджоникидзе, 56 фран
shop_id                                  0
Name: 0, dtype: object

## 0.3. Helper Functions

# 1. Data Description

## 1.1. Merging Tables

In [95]:
df1 = pd.merge(items,item_categories,on='item_category_id')
df1 = pd.merge(sales_train,df1,on='item_id')
df1 = pd.merge(df1,shops,on='shop_id')
df1.sample().T

,181785
date,19.04.2015
date_block_num,27
shop_id,27
item_id,6628
item_price,699
item_cnt_day,1
item_name,"Sniper Elite 3 [PC, русская версия]"
item_category_id,30
item_category_name,Игры PC - Стандартные издания
shop_name,"Москва ТЦ ""МЕГА Белая Дача II"""


## 1.2. Data Dimensions

In [96]:
print('Number of Rows: {:,}'.format(df1.shape[0]))
print('Number of Columns: ',df1.shape[1])

Number of Rows: 2,935,849
Number of Columns:  10


## 1.3. Data Types

In [97]:
df1.dtypes

date                   object
date_block_num          int64
shop_id                 int64
item_id                 int64
item_price            float64
item_cnt_day          float64
item_name              object
item_category_id        int64
item_category_name     object
shop_name              object
dtype: object

## 1.4. Change Data Types

In [98]:
df1['date'] = pd.to_datetime(df1['date'])

## 1.5. NA Checking

In [99]:
df1.isna().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
item_name             0
item_category_id      0
item_category_name    0
shop_name             0
dtype: int64

## 1.6. Fillout NA

- None of the columns have null values;

## 1.7. Descriptive Statistical

In [100]:
num_attributes = df1.select_dtypes(include=['int64','float64'])
cat_attributes = df1.select_dtypes(exclude=['int64','float64','datetime64[ns]'])

### 1.7.1. Numerical Attributes

In [101]:
# Central Tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# Contacatenate
n = pd.concat([d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
n.columns = ['Attributes','Min','Max','Range','Mean','Median','Std','Skew','Kurtosis']

In [102]:
n

,Attributes,Min,Max,Range,Mean,Median,Std,Skew,Kurtosis
0,date_block_num,0.0,33.0,33.0,14.569911,14.0,9.422986,0.203858,-1.082869
1,shop_id,0.0,59.0,59.0,33.001728,31.0,16.226970,-0.072361,-1.025358
2,item_id,0.0,22169.0,22169.0,10197.227057,9343.0,6324.296277,0.257174,-1.225210
3,item_price,-1.0,307980.0,307981.0,890.853233,399.0,1729.799336,10.750423,445.532826
4,item_cnt_day,-22.0,2169.0,2191.0,1.242641,1.0,2.618834,272.833162,177478.098774
5,item_category_id,0.0,83.0,83.0,40.001383,40.0,17.100756,0.318283,-0.525158


### 1.7.2. Categorical Attributes

In [103]:
# Unique values
cat_attributes.apply(lambda x: x.unique().shape[0])

item_name             21807
item_category_name       84
shop_name                60
dtype: int64

# 2. Feature Engineering

In [104]:
df2 = df1.copy()

## 2.1. Hypothesis Mind Map

## 2.2. List of Hypothesis

## 2.3. Feature Engineering

In [107]:
!pip install textblob
from textblob import TextBlob
blob = TextBlob('comment ca va ?')
blob.translate(to='en')

HTTPError: ignored

In [87]:
gs = goslate.Goslate()
gs.translate('Кино - Blu-Ray', 'en')

HTTPError: ignored

In [85]:
for string in list(df1['item_category_name'].unique()[:4]):
#primary_text = 'Кино - Blu-Ray'
  goslate.Goslate().translate(string, 'en')
  #print(string)

HTTPError: ignored

In [67]:
list(map(lambda x: gs.translate(x, 'en') ,df1['item_category_name'].unique()))

HTTPError: ignored

In [47]:
df1[df1['item_category_id']==0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
243013,2013-10-01,0,25,16257,148.0,1.0,Наушники RITMIX RH-124 Black,0,PC - Гарнитуры/Наушники,"Москва ТРК ""Атриум"""
1096804,2013-02-04,3,38,16256,22.4,1.0,Наушники RITMIX RH-120,0,PC - Гарнитуры/Наушники,"Омск ТЦ ""Мега"""
1379422,2013-02-28,1,0,16255,93.0,1.0,Наушники PHILIPS SBC HC8680,0,PC - Гарнитуры/Наушники,"!Якутск Орджоникидзе, 56 фран"
